In [1]:
import numpy as np
import json
from konlpy.tag import Okt
from tensorflow import keras
from pyjosa.josa import Josa

## 학습 데이터 확인 

In [2]:
f = open("file2/label.txt", "r", encoding="utf-8")
t = f.read().split('\n')
text1 = ''
for i in t :
    text1 = text1 + i
print('코퍼스의 길이: ', len(text1))
text1

코퍼스의 길이:  1134


'@가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 자동차를 택시를 자동차를 택시를 자동차를 택시를 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 아래 앞 오른쪽 왼쪽 위 아래 앞 오른쪽 왼쪽 위 아래 앞 오른쪽 왼쪽 위  공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 개를 개를 개를 물. 불이 음식이 물. 불이 음식이 물. 불이 음식이 안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘

#### AIHub(AI데이터) - 한국지능정보사회진흥원

* 출처: 한국어 생성 기반 상식추론 데이터셋

In [3]:
import pandas as pd
train = pd.read_csv('AI hub/train_data.csv')
train['reference_1'].head(10000)

0                    야구 선수가 공을 쳤다.
1                   거리에 버스와 차가 있다.
2                 칼과 당근이 도마 위에 있다.
3             보트 한척이 물 위를 항해하고 있다.
4                       편대 비행을 하다.
                   ...            
9995           세면대와 욕조 사이에 변기가 있다.
9996        가위와 선글라스가 셔츠 위에 놓여 있다.
9997      도시 철도 차량은 위의 선에 연결되어 있다.
9998         비행기가 하늘에서 급회전을 하고 있다.
9999    개 한마리가 얼음 물 위를 걷고 있는 것 같다.
Name: reference_1, Length: 10000, dtype: object

In [4]:
lit = train['reference_1'].head(10000).values.tolist()
lit

['야구 선수가 공을 쳤다.',
 '거리에 버스와 차가 있다.',
 '칼과 당근이 도마 위에 있다.',
 '보트 한척이 물 위를 항해하고 있다.',
 '편대 비행을 하다.',
 '해안에 배 두척이 정박해 있다.',
 '화장실에는 두개의 화장실이 있다.',
 '냉장고와 식탁이 있습니다.',
 '말과 기수는 장애물을 뛰어넘는다.',
 '두 기차가 나란히 주차되어 있다.',
 '코끼리가 물 속에 다른 코끼리 옆에 있다.',
 '키보드가 테이블 위에 있다.',
 '기차가 승강장 옆에 있다.',
 '트럭 두대가 나란히 주차되어 있다.',
 '방에는 소파와 테이블이 있다.',
 '기린이 울타리 안에 있다.',
 '건물 앞면에 시계가 있다.',
 '당당한 교회 옆에 시계가 있다.',
 '주차장 입구입니다.',
 '꽃들이 탁자 위 꽃병에 있다.',
 '거실 사진이 있다.',
 '야외 사진이 있다.',
 '동물원 안에는 매우 큰 갈색 곰이 있다.',
 '배와 오렌지에는 흙과 갈색 반점이 있다.',
 '통조림 병이 믹서기 위에 거꾸로 놓여 있다.',
 '소가 길을 따라 함께 이동한다는 소문이 있다.',
 '노란 자전거 표지판이 바로 뒤에 있다.',
 '눈 속의 개는 초록색 원반을 들고 있었다.',
 '초라하게 생긴 방에는 난로 종류가 있다.',
 '화장실에는 이중 화장지 통이 있다.',
 '작은 새들은 야외에 서서 물을 체크한다.',
 '길에 수컷 염소가 걷고 있다.',
 '소방차가 화재 앞에 주차되어 있다.',
 '녹색 시내 버스가 길로 들어온다.',
 '한 사람이 해변에서 용 연을 날리고 있다.',
 '한 야드에 두 사람이 연을 날리고 있다.',
 '날아다니는 연들 옆에 공룡 조각 상이 있다.',
 '벤치가 풀이 무성한 아스팔트 앞에 앉아 있다.',
 '소화전이 잔디밭 중앙에 있다.',
 '책장 옆 거실에 TV모니터가 있다.',
 '한쌍의 기린이 먼지 위에 서 있었다.',
 '기차 뒤에 작은 트럭이 있다.',
 '왼손이 스마트 폰을 들고 있다.',
 '몇명의 사람들이 코끼리

In [5]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()
text2 = listToString(lit)
print(text2)

야구 선수가 공을 쳤다. 거리에 버스와 차가 있다. 칼과 당근이 도마 위에 있다. 보트 한척이 물 위를 항해하고 있다. 편대 비행을 하다. 해안에 배 두척이 정박해 있다. 화장실에는 두개의 화장실이 있다. 냉장고와 식탁이 있습니다. 말과 기수는 장애물을 뛰어넘는다. 두 기차가 나란히 주차되어 있다. 코끼리가 물 속에 다른 코끼리 옆에 있다. 키보드가 테이블 위에 있다. 기차가 승강장 옆에 있다. 트럭 두대가 나란히 주차되어 있다. 방에는 소파와 테이블이 있다. 기린이 울타리 안에 있다. 건물 앞면에 시계가 있다. 당당한 교회 옆에 시계가 있다. 주차장 입구입니다. 꽃들이 탁자 위 꽃병에 있다. 거실 사진이 있다. 야외 사진이 있다. 동물원 안에는 매우 큰 갈색 곰이 있다. 배와 오렌지에는 흙과 갈색 반점이 있다. 통조림 병이 믹서기 위에 거꾸로 놓여 있다. 소가 길을 따라 함께 이동한다는 소문이 있다. 노란 자전거 표지판이 바로 뒤에 있다. 눈 속의 개는 초록색 원반을 들고 있었다. 초라하게 생긴 방에는 난로 종류가 있다. 화장실에는 이중 화장지 통이 있다. 작은 새들은 야외에 서서 물을 체크한다. 길에 수컷 염소가 걷고 있다. 소방차가 화재 앞에 주차되어 있다. 녹색 시내 버스가 길로 들어온다. 한 사람이 해변에서 용 연을 날리고 있다. 한 야드에 두 사람이 연을 날리고 있다. 날아다니는 연들 옆에 공룡 조각 상이 있다. 벤치가 풀이 무성한 아스팔트 앞에 앉아 있다. 소화전이 잔디밭 중앙에 있다. 책장 옆 거실에 TV모니터가 있다. 한쌍의 기린이 먼지 위에 서 있었다. 기차 뒤에 작은 트럭이 있다. 왼손이 스마트 폰을 들고 있다. 몇명의 사람들이 코끼리 뒤를 보고 있다. 밤 비행기, 밝은 활주로를 날다. 검정색 푸들이 소화전 쪽으로 기울어져 있다. 역에서 지하철 근처에 사람들이 있다. 교통 신호등은 눈에 거의 넘어질 뻔 한 후에도 계속 작동한다. 스테인리스 강 소화전 세개가 상점 앞에 있다. 족제비는 외딴 곳에 머리를 숙이고 있다. 주유소 로고가 밤을 밝히고

In [6]:
text =  text1 + text2
text

'@가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 가렵다.  냄새나다.  배고프다. 삼키다. 어지럽다. 자동차를 택시를 자동차를 택시를 자동차를 택시를 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 누나가 동생이 선생님이 어린이가 아빠가 언니가 엄마가 오빠가 친구가 형이 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 금요일에 내년에 내일은 목요일에 수요일에 어제. 오늘은 올해는 월요일에 일요일에 작년에 토요일에 화요일에 아래 앞 오른쪽 왼쪽 위 아래 앞 오른쪽 왼쪽 위 아래 앞 오른쪽 왼쪽 위  공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 공원에서 바다에서 병원에서 산에서 아파트에 집에서 학교에서 화장실에서 개를 개를 개를 물. 불이 음식이 물. 불이 음식이 물. 불이 음식이 안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  안녕하세요. 돕다. 슬프다. 타다. 내리다. 남자가 여자가 가다. 오다. 주다.  부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 부탁합니다. 감사합니다. 괜찮습니다. 미안합니다. 수고. 좋다. 싫다. 알다. 모르다. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘. 나는 너가 빨리. 있다. 없다. 맞다. 아니다. 끝. 잘

In [7]:
# 함수 1 : 딕셔너리에 데이터 등록
def make_dic(words):

    tmp = ["@"] 
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 1: continue 
        if len(tmp) > 2: tmp = tmp[1:]
        set_word(dic, tmp)        
    return dic

# 함수 2 : 딕셔너리에 데이터 등록
def set_word(dic, s2):
    w1, w2 = s2 
    
    if not w1 in dic:
        dic[w1] = {}
         
    if not w2 in dic[w1]:
        dic[w1][w2] = 1

In [8]:
from konlpy.tag import Okt

text = text.replace("…", "") 

Okt=Okt()
malist = Okt.pos(text, norm=True) 
words = []

for word in malist:
    if not word[1] in ["Punctuation"]:
        words.append(word[0])
    if word[0] == ".":
        words.append(word[0])
            # print(words)
dic = make_dic(words)
print(dic)
print(dic.keys())

{'@': {'가렵다': 1}, '가렵다': {'.': 1}, '.': {'냄새': 1, '배고프다': 1, '삼키다': 1, '어지럽다': 1, '가렵다': 1, '자동차': 1, '오늘': 1, '불': 1, '돕다': 1, '슬프다': 1, '타다': 1, '내': 1, '남자': 1, '오다': 1, '주다': 1, '안녕하세요': 1, '부탁': 1, '감사합니다': 1, '괜찮습니다': 1, '미안합니다': 1, '수고': 1, '좋다': 1, '싫다': 1, '알다': 1, '모르다': 1, '나': 1, '있다': 1, '없다': 1, '맞다': 1, '아니다': 1, '끝': 1, '잘': 1, '우리': 1, '수화': 1, '번역': 1, '반드시': 1, '사랑': 1, '무엇': 1, '야구': 1, '거리': 1, '칼': 1, '보트': 1, '편대': 1, '해안': 1, '화장실': 1, '냉장고': 1, '말': 1, '두': 1, '코끼리': 1, '키': 1, '기차': 1, '트럭': 1, '방': 1, '기린': 1, '건물': 1, '당당한': 1, '주차장': 1, '꽃들이': 1, '거실': 1, '야외': 1, '동물원': 1, '배': 1, '통조림': 1, '소가': 1, '노란': 1, '눈': 1, '초라하게': 1, '작은': 1, '길': 1, '소방차': 1, '녹색': 1, '한': 1, '날아다니는': 1, '벤치': 1, '소화전': 1, '책장': 1, '왼손': 1, '몇': 1, '밤': 1, '검정색': 1, '역': 1, '교통': 1, '스테인리스': 1, '족제비': 1, '주유소': 1, '어수선한': 1, '콘크리트': 1, '도심지': 1, '핫도그': 1, '자전거': 1, '곰': 1, '주차': 1, '스키': 1, '오렌지': 1, '서빙': 1, '이': 1, '파이': 1, '새': 1, '임시': 1, '닭': 1, '빵': 1, '개': 1, '테니스': 1, '다

## 학습데이터 정제

In [9]:
chars = sorted(list(set(dic.keys()))) # 정렬
print('사용되고 있는 문자의 수:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자
    
print(char_indices)
#print(indices_char)

사용되고 있는 문자의 수: 5468
{'.': 0, '@': 1, 'AC': 2, 'BMW': 3, 'CD': 4, 'DJ': 5, 'DVD': 6, 'L': 7, 'MTU': 8, 'PC': 9, 'SUV': 10, 'TV': 11, 'USB': 12, 'Wii': 13, 'l': 14, 'mlb': 15, 'tv': 16, 'v': 17, 'vcr': 18, 'wadsworthst': 19, 'wii': 20, '가': 21, '가가': 22, '가게': 23, '가격': 24, '가격표': 25, '가고': 26, '가구': 27, '가구점': 28, '가까운': 29, '가까이': 30, '가깝다': 31, '가깝습니다': 32, '가느다란': 33, '가는': 34, '가는지': 35, '가다': 36, '가드레일': 37, '가득': 38, '가득하다': 39, '가득한': 40, '가득했고': 41, '가라앉는다': 42, '가라앉다': 43, '가라앉아': 44, '가라앉힌다': 45, '가려고': 46, '가려져': 47, '가렵다': 48, '가렸다': 49, '가로': 50, '가로등': 51, '가로막': 52, '가로수': 53, '가로지르고': 54, '가로지르는': 55, '가로질러': 56, '가루': 57, '가르치고': 58, '가른다': 59, '가를': 60, '가리고': 61, '가리려고': 62, '가리며': 63, '가리비': 64, '가리켰다': 65, '가리키고': 66, '가린다': 67, '가만히': 68, '가미': 69, '가방': 70, '가볍게': 71, '가상': 72, '가스': 73, '가슴': 74, '가에': 75, '가에서': 76, '가옥': 77, '가운': 78, '가운데': 79, '가위': 80, '가을': 81, '가의': 82, '가이드': 83, '가장': 84, '가장자리': 85, '가재': 86, '가전': 87, '가정': 88, '가제': 89, '가져가고': 90, '가

In [13]:
sentences = []
next_chars = []
maxlen = 20

for i in range(len(words)-1):
    sentences.append(words[i])
    next_chars.append(words[i + 1])

print('학습할 구문의 수:', len(sentences))
# print(sentences)
# print(next_chars)

학습할 구문의 수: 104227


In [ ]:
print(len(sentences), len(chars))

## 모델

In [14]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i in range(len(sentences)) :  
    X[ i, len(sentences[i]), char_indices[sentences[i]] ] = 1. 
    y[ i, char_indices[next_chars[i]] ] = 1

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [16]:
from tensorflow import keras
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from tensorflow.keras import optimizers

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# 모델 구축하기(LSTM)
print('모델1을 구축합니다...')

keras.backend.clear_session()

model1 = Sequential()

model1.add(LSTM(128, input_shape=(maxlen, len(chars))))

model1.add(Dense(len(chars)))

model1.add(Activation('softmax'))

optimizer = optimizers.RMSprop(learning_rate=0.01)

model1.compile(loss='categorical_crossentropy', optimizer=optimizer)
model1.summary()

모델1을 구축합니다...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               2865664   
                                                                 
 dense (Dense)               (None, 5468)              705372    
                                                                 
 activation (Activation)     (None, 5468)              0         
                                                                 
Total params: 3,571,036
Trainable params: 3,571,036
Non-trainable params: 0
_________________________________________________________________


In [17]:
# model1.fit(x_train, y_train, batch_size=128, epochs=1000, verbose=1) #nb_epoch=1) # nb는 옛날 버전

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0, 
                   patience=5,  
                   verbose=1,   
                   restore_best_weights=True) 

In [ ]:
# 모델 학습
history = model1.fit(x_train, y_train,epochs=10,
                     verbose=1, validation_split=0.2,
                     batch_size=64, callbacks=[es])

Epoch 1/10
912/912 [==============================] - 303s 329ms/step - loss: 5.3075 - val_loss: 5.0437
Epoch 2/10
202/912 [=====>........................] - ETA: 3:24 - loss: 4.8326

In [21]:
y_pred = model1.predict(x_test, verbose=1)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred)**(1/2)) # 오차제곱의 평균(MSE) 의 루트
print('MAE: ',mean_absolute_error(y_test, y_pred)) 
print('R2: ', r2_score(y_test, y_pred)) #  결과 값 정도의 오류가 있는 것,1에 가까울수록 클수록 좋음

978/978 [==============================] - 89s 90ms/step
MSE:  0.00014350472
RMSE:  0.01197934539558362
MAE:  0.00027964727
R2:  0.0012805255321666218


## 모델 테스트

In [26]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
def josa_fuc(c, nextc):
    try:
        return c + Josa.get_josa(c, nextc)
    except :
        return c + nextc    

In [36]:
def predict_mo(lst):
    result =''
    
    for i in lst:
        # print(i)
        if char_indices.get(i) == None or i[-1] == '요' or i[-1] == '다': 
            # 입력값이 모델학습데이터에 없거나 한문장 또는 마지막 말일 때
            result += i + ' '
            continue
        
        if i[-1] == '끝':
            if result[-2] == '다':
                result = result[:-2]
                result += '었다 '
            continue
        
            
        x = np.zeros((1, 20, len(chars)))
        x[0, len(i), char_indices[i]] = 1.

        preds = model1.predict(x, verbose=0)[0]
        next_index = sample(preds)
        next_char = indices_char[next_index]
        
        if next_char == '.':
            result += i + ' '
            continue
            
        y = josa_fuc(i, next_char)
        result += y + ' '
        
    return result

In [51]:
lst = ['안녕하세요','우리','2','단체','수화','번역','만들다', '끝' , '잘', '부탁합니다']

sentence = predict_mo(lst)
print(sentence)

안녕하세요 우리가 2 단체하얀 수화 번역을 만들었다 잘여러 부탁합니다 


In [41]:
lst = ['나','집','개','키우다']
print(predict_mo(lst))

나열차 집에 개가 키우다 


In [42]:
lst = ['나','배고프다']
print(predict_mo(lst))

나있다 배고프다 


In [43]:
lst = ['나','어지럽다']
print(predict_mo(lst))

나가 어지럽다 


In [44]:
lst = ['음식','냄새나다']
print(predict_mo(lst))

음식을 냄새나다 
